# **Installation**

In [1]:
import resampy

In [2]:
!pip install audb


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
!pip install shap


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
!pip install opensmile


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
!pip install xgboost


[notice] A new release of pip is available: 25.0 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [6]:
import os
import time
import numpy as np
import pandas as pd
import seaborn as sns
import audb
import audiofile
from xgboost import XGBClassifier
import opensmile
from sklearn.utils import shuffle
import random
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score, roc_curve, auc, confusion_matrix, log_loss, f1_score, accuracy_score, precision_score, recall_score
import matplotlib.pyplot as plt
%matplotlib inline


# Code

In [7]:
smile = opensmile.Smile(
    feature_set=opensmile.FeatureSet.ComParE_2016,
    feature_level=opensmile.FeatureLevel.Functionals,
)

In [8]:
def load_data_from_folds(directory):
    data = {"HS": [], "PD": []}
    
    # Load each fold
    for fold in range(1, 11):
        fold_path = os.path.join(directory, f"fold{fold}")
        hs_folder = os.path.join(fold_path, "HS")
        pd_folder = os.path.join(fold_path, "PD")

        hs_files = [os.path.join(hs_folder, f) for f in os.listdir(hs_folder) if f.endswith(".wav")]
        pd_files = [os.path.join(pd_folder, f) for f in os.listdir(pd_folder) if f.endswith(".wav")]

        data["HS"].append(hs_files)
        data["PD"].append(pd_files)
    
    return data

In [ ]:
# #Test
# directory = r'C:\Users\Hp\Desktop\Lecture Files\DH604\Publication\datasets'
# data_ = load_data_from_folds(directory)

In [ ]:
# data_['HS'][1]

['C:\\Users\\Hp\\Desktop\\Lecture Files\\DH604\\Publication\\datasets\\fold2\\HS\\B1AGNIGNEE54F230320171018.wav',
 'C:\\Users\\Hp\\Desktop\\Lecture Files\\DH604\\Publication\\datasets\\fold2\\HS\\B1APGANRET55F170320171104.wav',
 'C:\\Users\\Hp\\Desktop\\Lecture Files\\DH604\\Publication\\datasets\\fold2\\HS\\B2AGNIGNEE54F230320171019.wav',
 'C:\\Users\\Hp\\Desktop\\Lecture Files\\DH604\\Publication\\datasets\\fold2\\HS\\B2APGANRET55F170320171105.wav']

In [ ]:
# data_['PD'][5]

['C:\\Users\\Hp\\Desktop\\Lecture Files\\DH604\\Publication\\datasets\\fold6\\PD\\B1MBAUROIN45M100220171003.wav',
 'C:\\Users\\Hp\\Desktop\\Lecture Files\\DH604\\Publication\\datasets\\fold6\\PD\\B1MCIICLHL46M240120171825.wav',
 'C:\\Users\\Hp\\Desktop\\Lecture Files\\DH604\\Publication\\datasets\\fold6\\PD\\B2MBAUROIN45M100220171006.wav',
 'C:\\Users\\Hp\\Desktop\\Lecture Files\\DH604\\Publication\\datasets\\fold6\\PD\\B2MCIICLHL46M240120171827.wav']

In [9]:
def process_audio_files(files, label, duration=10):
    results = []
    for file_path in files:
        if file_path.endswith(".wav"):
            try:
                signal, sampling_rate = audiofile.read(
                    file_path,
                    duration=duration,
                    always_2d=True
                )
                # print(sampling_rate)
                features = smile.process_signal(signal, sampling_rate)
                features.insert(0, 'file_name', file_path)
                features['Y'] = label
                results.append(features)
            except Exception as e:
                print(f"Error processing {file_path}: {e}")
    
    df_results = pd.concat(results, ignore_index=True)
    # print(f"Done processing {len(results)} files.")
    return df_results


In [5]:
# def process_audio_files(files, label, duration=3, target_sr=16000):
#     results = []
#     for file_path in files:
#         if file_path.endswith(".wav"):
#             try:
#                 signal, sampling_rate = audiofile.read(
#                     file_path,
#                     duration=duration,
#                     always_2d=True
#                 )
                
#                 if sampling_rate != target_sr:
#                     signal = resampy.resample(signal[:, 0], sampling_rate, target_sr)
#                     signal = signal.reshape(-1, 1)  
#                     sampling_rate = target_sr

#                 print(f"{file_path} -> Resampled to {sampling_rate} Hz")

#                 features = smile.process_signal(signal, sampling_rate)
#                 features.insert(0, 'file_name', file_path)
#                 features['Y'] = label
#                 results.append(features)

#             except Exception as e:
#                 print(f"Error processing {file_path}: {e}")

#     df_results = pd.concat(results, ignore_index=True)
#     return df_results

In [10]:
def drop_na(X, y):
    valid_indices = X.dropna(axis=0).index
    X = X.loc[valid_indices]
    y = y.loc[valid_indices]
    return X, y

In [11]:
def get_train_test_data(fold_number, data):
    test_hs = data["HS"][fold_number-1]
    test_pd = data["PD"][fold_number-1]

    # Combine the remaining folds for training
    train_hs = [item for i, item in enumerate(data["HS"]) if i != fold_number-1]
    train_pd = [item for i, item in enumerate(data["PD"]) if i != fold_number-1]

    # Flatten lists for training
    train_hs = [item for sublist in train_hs for item in sublist]
    train_pd = [item for sublist in train_pd for item in sublist]

    # Process the files
    X_train_hs = process_audio_files(train_hs, label=0)
    X_train_pd = process_audio_files(train_pd, label=1)
    
    X_test_hs = process_audio_files(test_hs, label=0)
    X_test_pd = process_audio_files(test_pd, label=1)

    # Combine HS and PD for train and test
    X_train = pd.concat([X_train_hs, X_train_pd], ignore_index=True)
    X_test = pd.concat([X_test_hs, X_test_pd], ignore_index=True)

    y_train = X_train["Y"]
    X_train.drop(columns=["Y",'file_name'], inplace=True)

    y_test = X_test["Y"]
    X_test.drop(columns=["Y",'file_name'], inplace=True)

    # Shuffle the train and test sets
    X_train, y_train = shuffle(X_train, y_train, random_state=42)
    X_test, y_test = shuffle(X_test, y_test, random_state=42)

    # Drop NA
    X_train, y_train = drop_na(X_train, y_train)
    X_test, y_test = drop_na(X_test, y_test)

    # Scaling the data
    scaler = StandardScaler()
    X_train = pd.DataFrame(scaler.fit_transform(X_train), columns=X_train.columns, index=X_train.index)
    X_test = pd.DataFrame(scaler.transform(X_test), columns=X_test.columns, index=X_test.index)

    return X_train, y_train, X_test, y_test


In [ ]:
# X_train, y_train, X_test, y_test = get_train_test_data(1, data_)

In [ ]:
# print("X_train shape:", X_train.shape)
# print("y_train shape:", y_train.shape)
# print("X_test shape:", X_test.shape)
# print("y_test shape:", y_test.shape)

X_train shape: (72, 6373)
y_train shape: (72,)
X_test shape: (8, 6373)
y_test shape: (8,)


In [ ]:
# X_train.head(2)

,audspec_lengthL1norm_sma_range,audspec_lengthL1norm_sma_maxPos,audspec_lengthL1norm_sma_minPos,audspec_lengthL1norm_sma_quartile1,audspec_lengthL1norm_sma_quartile2,audspec_lengthL1norm_sma_quartile3,audspec_lengthL1norm_sma_iqr1-2,audspec_lengthL1norm_sma_iqr2-3,audspec_lengthL1norm_sma_iqr1-3,audspec_lengthL1norm_sma_percentile1.0,...,mfcc_sma_de[14]_peakRangeAbs,mfcc_sma_de[14]_peakRangeRel,mfcc_sma_de[14]_peakMeanAbs,mfcc_sma_de[14]_peakMeanMeanDist,mfcc_sma_de[14]_peakMeanRel,mfcc_sma_de[14]_minRangeRel,mfcc_sma_de[14]_meanRisingSlope,mfcc_sma_de[14]_stddevRisingSlope,mfcc_sma_de[14]_meanFallingSlope,mfcc_sma_de[14]_stddevFallingSlope
4,-0.503861,-0.534324,-0.600771,-0.932519,-0.725894,-0.540846,-0.466143,-0.318095,-0.387454,-0.275918,...,-0.692934,-0.174968,-0.382107,-0.462144,1.029679,0.018335,0.091457,-0.572677,0.338687,-1.237910
62,-0.018046,-0.983622,-1.347266,0.272143,0.288131,0.296660,0.251986,0.280493,0.277985,-0.165103,...,0.580623,-0.247839,0.890351,0.918647,-0.952045,1.511754,1.270072,0.141734,0.612055,0.251577


In [ ]:
# y_train.head()

4     0
62    1
18    0
0     0
28    0
Name: Y, dtype: int64

In [3]:
def create_model(model_type):
    if model_type == 'random_forest':
        # model = RandomForestClassifier(random_state=42)
        model = RandomForestClassifier(max_depth=5, max_features=10, min_samples_leaf=10, min_samples_split=5, n_estimators=100, random_state=42)

    elif model_type == 'logistic_regression':
        # model = LogisticRegression(random_state=42)
        model =LogisticRegression(C=0.1, max_iter=1000, penalty='l2', solver='liblinear', random_state=42)

    elif model_type == 'svm':
        model = SVC(random_state=42, probability=True)
        # model = SVC(C=0.2 ,class_weight=None , gamma=0.01, kernel='rbf', shrinking=True, random_state=42, probability=True)

    elif model_type == 'xgboost':
        # model = XGBClassifier(random_state=42)
        model = XGBClassifier(colsample_bytree=0.8, gamma=0.1, learning_rate=0.05, max_depth=5, min_child_weight=3, n_estimators=50, reg_alpha=0.1, reg_lambda=1, subsample=0.8)

    else:
        raise ValueError("Invalid model type. Choose from 'random_forest', 'logistic_regression', 'svm', or 'xgboost'.")
    return model

In [4]:
def evaluate_model(X_train, y_train, X_test, y_test, model_type):

    if model_type == 'xgboost':
        for X in [X_train, X_test]:
            X.columns = [col.replace('[', '(').replace(']', ')').replace('<', '_less_than_') for col in X.columns]

    model = create_model(model_type)
    
    model.fit(X_train, y_train)
    
    # Get predictions
    y_train_pred = model.predict(X_train)
    y_test_pred = model.predict(X_test)
    
    # Metrics
    accuracy_train = accuracy_score(y_train, y_train_pred)
    accuracy_test = accuracy_score(y_test, y_test_pred)
    
    roc_auc_train = roc_auc_score(y_train, model.predict_proba(X_train)[:, 1])
    roc_auc_test = roc_auc_score(y_test, model.predict_proba(X_test)[:, 1])
    
    f1_train = f1_score(y_train, y_train_pred)
    f1_test = f1_score(y_test, y_test_pred)
    
    precision_train = precision_score(y_train, y_train_pred)
    precision_test = precision_score(y_test, y_test_pred)
    
    recall_train = recall_score(y_train, y_train_pred)
    recall_test = recall_score(y_test, y_test_pred)
    
    train_loss = log_loss(y_train, model.predict_proba(X_train)[:, 1])
    test_loss = log_loss(y_test, model.predict_proba(X_test)[:, 1])

    return {
        "accuracy_train": accuracy_train,
        "accuracy_test": accuracy_test,
        "roc_auc_train": roc_auc_train,
        "roc_auc_test": roc_auc_test,
        "f1_train": f1_train,
        "f1_test": f1_test,
        "precision_train": precision_train,
        "precision_test": precision_test,
        "recall_train": recall_train,
        "recall_test": recall_test,
        "train_loss": train_loss,
        "test_loss": test_loss
    }


In [5]:
# MIQ

selected_features = [
    "audspec_lengthL1norm_sma_maxPos",
    "pcm_RMSenergy_sma_kurtosis",
    "audspecRasta_lengthL1norm_sma_de_upleveltime75",
    "pcm_RMSenergy_sma_de_risetime",
    "pcm_zcr_sma_de_leftctime",
    "audSpec_Rfilt_sma[3]_lpc4",
    "audSpec_Rfilt_sma[6]_quartile1",
    "audSpec_Rfilt_sma[6]_percentile1.0",
    "audSpec_Rfilt_sma[13]_percentile99.0",
    "audSpec_Rfilt_sma[18]_maxPos",
    "audSpec_Rfilt_sma[23]_kurtosis",
    "pcm_fftMag_psySharpness_sma_upleveltime50",
    "mfcc_sma[1]_lpc3",
    "mfcc_sma[4]_segLenStddev",
    "mfcc_sma[10]_meanSegLen",
    "audSpec_Rfilt_sma_de[0]_iqr1-2",
    "audSpec_Rfilt_sma_de[6]_lpc2",
    "audSpec_Rfilt_sma_de[13]_lpc2",
    "pcm_fftMag_spectralEntropy_sma_de_upleveltime90",
    "mfcc_sma_de[2]_upleveltime25",
    "mfcc_sma_de[3]_percentile1.0",
    "mfcc_sma_de[6]_iqr1-2",
    "mfcc_sma_de[8]_quartile1",
    "F0final_sma_percentile1.0",
    "voicingFinalUnclipped_sma_skewness",
    "jitterLocal_sma_flatness",
    "jitterLocal_sma_de_lpc4",
    "mfcc_sma[4]_meanRisingSlope",
    "mfcc_sma[14]_stddevFallingSlope",
    "pcm_fftMag_spectralEntropy_sma_de_stddevFallingSlope"
]

In [6]:
def main(directory, model_type="random_forest", feature_list=True):
    data = load_data_from_folds(directory)
    
    metrics = []
    
    for fold_number in range(1, 11):
        print(f"Evaluating fold {fold_number}...")
        
        X_train, y_train, X_test, y_test = get_train_test_data(fold_number, data)
        
        if feature_list:
            X_train = X_train[selected_features]
            X_test = X_test[selected_features]
        
        print("X_train shape:", X_train.shape)
        print("y_train shape:", y_train.shape)
        print("X_test shape:", X_test.shape)
        print("y_test shape:", y_test.shape)
        
        fold_metrics = evaluate_model(X_train, y_train, X_test, y_test, model_type)
        metrics.append(fold_metrics)
        
        print(f"Metrics for fold {fold_number}: {fold_metrics}")
        print(" ")
    
    # Calculate average and standard deviation
    avg_metrics = {metric: np.mean([fold[metric] for fold in metrics]) for metric in metrics[0]}
    std_metrics = {metric: np.std([fold[metric] for fold in metrics]) for metric in metrics[0]}
    
    print("\nAverage Metrics:")
    print(avg_metrics)
    
    print("\nStandard Deviation of Metrics:")
    print(std_metrics)


In [7]:

# Specify the path to the dataset directory
directory = r"C:\Users\Hp\Desktop\Lecture Files\DH604\Publication\data\datasets_16k"

In [37]:
# # Call the main function with your model type
# main(directory, model_type="logistic_regression", feature_list = True)

In [8]:
# Call the main function with your model type
main(directory, model_type="svm", feature_list = True)

NameError: name 'load_data_from_folds' is not defined

In [59]:
# # Call the main function with your model type
# main(directory, model_type="random_forest", feature_list = True)

In [ ]:
# # Call the main function with your model type
# main(directory, model_type="xgboost", feature_list = True)

Evaluating fold 1...
X_train shape: (70, 30)
y_train shape: (70,)
X_test shape: (8, 30)
y_test shape: (8,)
Metrics for fold 1: {'accuracy_train': 1.0, 'accuracy_test': 1.0, 'roc_auc_train': 1.0, 'roc_auc_test': 1.0, 'f1_train': 1.0, 'f1_test': 1.0, 'precision_train': 1.0, 'precision_test': 1.0, 'recall_train': 1.0, 'recall_test': 1.0, 'train_loss': 0.11744242028875058, 'test_loss': 0.12198742747411101}
 
Evaluating fold 2...
X_train shape: (70, 30)
y_train shape: (70,)
X_test shape: (8, 30)
y_test shape: (8,)
Metrics for fold 2: {'accuracy_train': 1.0, 'accuracy_test': 1.0, 'roc_auc_train': 1.0, 'roc_auc_test': 1.0, 'f1_train': 1.0, 'f1_test': 1.0, 'precision_train': 1.0, 'precision_test': 1.0, 'recall_train': 1.0, 'recall_test': 1.0, 'train_loss': 0.11837825894574938, 'test_loss': 0.11421343424927329}
 
Evaluating fold 3...
X_train shape: (70, 30)
y_train shape: (70,)
X_test shape: (8, 30)
y_test shape: (8,)
Metrics for fold 3: {'accuracy_train': 1.0, 'accuracy_test': 1.0, 'roc_auc_tr